In [1]:
from pathlib import Path

In [2]:
nem = Path(Path.home(), 'nem-data', 'demand')

In [3]:
nem.glob('*DISPATCHREGIONSUM.csv')

<generator object Path.glob at 0x119585750>

In [4]:
paths = [p for p in nem.glob('**/*DISPATCHREGIONSUM.csv')]
paths

[PosixPath('/Users/adam/nem-data/demand/2013-11/2013-11/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2017-10/2017-10/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2017-11/2017-11/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2013-10/2013-10/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2011-8/2011-8/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2013-5/2013-5/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2016-11/2016-11/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2013-2/2013-2/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2019-5/2019-5/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2011-6/2011-6/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2012-10/2012-10/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/2019-2/2019-2/DISPATCHREGIONSUM.csv'),
 PosixPath('/Users/adam/nem-data/demand/

In [5]:
def process_demand(csv_path):
    print(csv_path)
    d = pd.read_csv(csv_path, index_col=0)
    columns = ['SETTLEMENTDATE', 'TOTALDEMAND', 'AVAILABLEGENERATION', 'INTERVENTION']

    d = d.loc[d['REGIONID'] == 'SA1', columns]
    d.loc[:, 'SETTLEMENTDATE'] = pd.to_datetime(d.loc[:, 'SETTLEMENTDATE'])
    d.loc[:, 'start-of-interval'] = d.loc[:, 'SETTLEMENTDATE'] - pd.Timedelta('5min')
    return d

demands = [process_demand(p) for p in paths]

/Users/adam/nem-data/demand/2013-11/2013-11/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2017-10/2017-10/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2017-11/2017-11/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2013-10/2013-10/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2011-8/2011-8/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2013-5/2013-5/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2016-11/2016-11/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2013-2/2013-2/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2019-5/2019-5/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2011-6/2011-6/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2012-10/2012-10/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2019-2/2019-2/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2011-1/2011-1/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2017-3/2017-3/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/2017-4/2017-4/DISPATCHREGIONSUM.csv
/Users/adam/nem-data/demand/

In [6]:
data = pd.concat(demands, axis=0)

In [7]:
data = data.sort_values('SETTLEMENTDATE', ascending=True)

In [8]:
idx = pd.date_range(data.loc[:, 'SETTLEMENTDATE'].iloc[0], data.loc[:, 'SETTLEMENTDATE'].iloc[-1], freq='5min')

In [9]:
len(idx)

1069632

In [10]:
len(data)

1112680

In [11]:
sum(data.duplicated('SETTLEMENTDATE', keep='last'))

43048

In [12]:
len(data) - len(idx)

43048

In [13]:
clean = data.loc[~data.duplicated(['SETTLEMENTDATE', 'INTERVENTION'], keep='last'), :]

In [14]:
len(clean) - len(idx)

41032

In [15]:
clean.head()

,SETTLEMENTDATE,TOTALDEMAND,AVAILABLEGENERATION,INTERVENTION,start-of-interval
I,,,,,
D,2009-07-01 00:05:00,1591.33,2985.610,0.0,2009-07-01 00:00:00
D,2009-07-01 00:10:00,1589.66,3004.164,0.0,2009-07-01 00:05:00
D,2009-07-01 00:15:00,1609.87,3012.470,0.0,2009-07-01 00:10:00
D,2009-07-01 00:20:00,1594.49,3003.234,0.0,2009-07-01 00:15:00
D,2009-07-01 00:25:00,1582.01,2987.409,0.0,2009-07-01 00:20:00


In [16]:
clean.to_csv('demand.csv')